## Import lbraries
---
This cell calls the libraries needed for computation, for downloading the data, and for plotting it.

In [1]:
# Numerical libraries:
import numpy as np
import pandas as pd
from scipy import interpolate
from scipy.optimize import curve_fit

# File libraries:
import csv

#Pyspedas libraries:
import pyspedas
import pytplot
from pytplot import del_data, get_data, store_data, tplot_options, tplot_names, tplot

#For handling timestamps:
import datetime as dt
from datetime import datetime
from datetime import timedelta

#Plotting libraries:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import ticker

## Import the list of events
---
This cell first opens the CSV file that contains the event list. It then creates a variable with the start times, stop times, and the probe to be used for each measurement.

In [2]:
starts = []
stops = []
mms_probe = []
themis_probe = []
artemis_probe = []
time_ranges = []

with open('../eventlist/eventlist.csv', newline='') as events:
    rows = csv.reader(events)
    next(rows)
    for r in rows:
        starts.append(dt.datetime.strptime(r[0], '%Y-%m-%dT%H:%M:%S.%fZ'))
        stops.append(dt.datetime.strptime(r[1], '%Y-%m-%dT%H:%M:%S.%fZ'))
        #mms_probe.append(r[2])
        themis_probe.append(str(r[3]))
        #artemis_probe.append(r[4])

for i, j in enumerate(starts):
    time_ranges.append([starts[i], stops[i]])

## Single-event analysis
---
The following cells perform the analysis for a single period of time. Please refer to the list of events for the correct index corresponding to the event you wish to analyze

The current variables are:
- `themis_fgm_data`: dictionary of parameters with the following relevant flags:
    - `'thx_fgs_gse'`: fluxgate magnetometer in GSE coords (replace x with spacecraft letter for the event)
        - `'x'`: the timestamps, in Unix time
        - `'y'`: the 3xn array of x, y, and z readings of the IMF

- `omni_mag_data`: dictionary of parameters with the following relevant flags:
    - `'BX_GSE'`: interplanetary magnetic field measurements in X
        - `'x'`: the timestamps, as a DateTime obj
        - `'y'`: the measurements of IMF Bx
    - `'BY_GSE'`: interplanetary magnetic field measurements in Y
        - `'x'`: the timestamps, as a DateTime obj
        - `'y'`: the measurements of IMF By
    - `'BZ_GSE'`: interplanetary magnetic field measurements in Z
        - `'x'`: the timestamps, as a DateTime obj
        - `'y'`: the measurements of IMF Bz

In [12]:
#-Choose input time range and spacecraft ID here:-----
event = 13
ID = themis_probe[event]
#-----------------------------------------------------

In [ ]:
omni_mag_data = pyspedas.omni.data(trange=starts[event].strftime('%Y-%m-%d/%H:%M:%S'),stops[event].strftime('%Y-%m-%d/%H:%M:%S')], datatype='1min', level='hro2', time_clip=True, notplot=True)
themis_fgm_data = pyspedas.themis.fgm(trange=[(starts[event]+timedelta(minutes=30)).strftime('%Y-%m-%d/%H:%M:%S'),stops[event].strftime('%Y-%m-%d/%H:%M:%S')], probe=themis_probe[event], varnames='th'+ID+'_fgs_gse', notplot=True)
themis_esa_data = pyspedas.themis.esa(trange=[(starts[event]+timedelta(minutes=30)).strftime('%Y-%m-%d/%H:%M:%S'),stops[event].strftime('%Y-%m-%d/%H:%M:%S')], probe=themis_probe[event], notplot=True)

This cell then creates a dataframe out of everything so that it can be interpolated and averaged eventually

In [388]:
tt = []
[tt.append(dt.datetime.utcfromtimestamp(themis_fgm_data['th'+ID+'_fgs_gse']['x'][i])) for i in range(len(themis_fgm_data['th'+ID+'_fgs_gse']['x']))]
themis_b = pd.DataFrame(np.column_stack((tt, themis_fgm_data['th'+ID+'_fgs_gse']['y'])))
omni_b = pd.DataFrame(np.column_stack((omni_mag_data['BX_GSE']['x'], pd.DataFrame(np.column_stack((omni_mag_data['BX_GSE']['y'], omni_mag_data['BY_GSE']['y'], omni_mag_data['BZ_GSE']['y']))).interpolate(method='linear'))))

Next, define the function for averaging the THEMIS data

In [389]:
def average(times, data):
    minute = times[0].minute # Set the current first minute of the data set
    timeAvgs = [] # Create empty array to store the time-averaged values
    avgArr = [] # Create empty storage array
    timeStep = [] # Create empty time step array
    for i in range(len(times)): # Index the values
        if times[i].minute == minute: # If the time of the next value equals the one set for the minute
            avgArr.append(data[i]) # Append this to the storage array
        elif times[i].minute == minute + 1: # If the time of the next value equals the next minute
            #print(avgArr)
            timeAvgs.append(np.average(avgArr)) # Average the storage array and append it to the time-averaged value array
            #print(np.average(avgArr))
            timeStep.append(dt.datetime(times[i-1].year, times[i-1].month, times[i-1].day, times[i-1].hour, times[i-1].minute, 00)) # Create a timestamp for the previous minute centered at 0s to line up with the orbit timestamps
            minute= times[i].minute # Set the new current minute to start averaging over
            avgArr = [] # Clear the storage array
        elif times[i].minute == minute - 59: # This is for rollover: when the next minute is 0
            #print(avgArr)
            timeAvgs.append(np.average(avgArr))
            #print(np.average(avgArr))
            timeStep.append(dt.datetime(times[i-1].year, times[i-1].month, times[i-1].day, times[i-1].hour, times[i-1].minute, 00))
            minute = times[i].minute
            avgArr = []
    return pd.DataFrame(np.column_stack((timeStep, timeAvgs))) # Return the time-averaged array and the timestamp array

In [390]:
a=average(themis_b[0], themis_b[1])

## Multi-even analysis
---
Stuff

In [ ]:
themis_fgm_data = {}
keys = range(len(time_ranges))
for i in keys:
    themis_fgm_data[i] = get_data(pyspedas.themis.fgm(trange=time_ranges[i], probe=themis_probe[i])[2])

In [92]:
themis_fgm_times = {}
themis_fgm_fields = {}
keys = range(len(themis_fgm_data[0][0]))

for i in keys:
    themis_fgm_times[i] = [themis_timestamps.append(dt.datetime.utcfromtimestamp(themis_fgm_data[0][j])) for j in range(len(themis_fgm_data[0]))]

array([1.31941472e+09, 1.31941473e+09, 1.31941473e+09, ...,
       1.31950083e+09, 1.31950084e+09, 1.31950084e+09])

In [82]:

themis_timestamps=[]


themis_fields = themis_fgm_data[1]

21-Feb-23 11:01:02: Downloading remote index: http://themis.ssl.berkeley.edu/data/themis/thd/l2/fgm/2011/
21-Feb-23 11:01:03: File is current: themis_data/thd/l2/fgm/2011/thd_l2_fgm_20111024_v01.cdf
